In [1]:
from transformers.models.mllama.modeling_mllama import MllamaVisionModel
from PIL import Image
from transformers import AutoProcessor
from astropy.table import Table
from tqdm.notebook import tqdm, trange
import os

In [2]:
# read all csv table
csv_data = Table.read("/mnt/data/CVPR2025/task1_data/test_no_classification.hdf5")
csv_data.keys()
print(csv_data["TARGETID"].shape, type(csv_data["TARGETID"]))

(21051,) <class 'astropy.table.column.Column'>


In [3]:
import torch.nn as nn
import torch
from safetensors import safe_open
# load a vision model
checkpoint="/mnt/data/CVPR2025/task1_data/Llama-3.2-11B-Vision-Instruct"
model = MllamaVisionModel.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)
projector = nn.Linear(7680, 4096, bias=True)
# state_dict = torch.load("/mnt/data/CVPR2025/task1_data/Llama-3.2-11B-Vision-Instruct/original/consolidated.pth", weights_only=True)
with safe_open("/mnt/data/CVPR2025/task1_data/Llama-3.2-11B-Vision-Instruct/model-00005-of-00005.safetensors", framework="pt",  device=0) as f:
    weight = f.get_tensor("multi_modal_projector.weight")
    bias = f.get_tensor("multi_modal_projector.bias")

# 初始化projector的权重和偏置
with torch.no_grad():
    projector.weight.copy_(weight)
    projector.bias.copy_(bias)

# 将projector移到GPU并设置为评估模式
projector = projector.cuda().eval()

# 如果需要和model一样使用bf16
projector = projector.bfloat16()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
image_root_dir = "/mnt/data/CVPR2025/task1_data/images/images"
all_image_paths = []
for id_ in tqdm(csv_data["TARGETID"]):
    img_path = os.path.join(image_root_dir, "{}.png".format(id_))
    all_image_paths.append(img_path)
    # inputs = processor(images=test_image, return_tensors="pt")
    # with torch.no_grad():
    #     for k in inputs:
    #         inputs[k] = inputs[k].cuda()
    #     output = model(**inputs)
    #     all_features.append(output[0].cpu().squeeze(0).squeeze(1))

  0%|          | 0/21051 [00:00<?, ?it/s]

In [16]:
import torch
# test image inputs
test_image_paths = all_image_paths[:8]
test_images = [Image.open(x) for x in test_image_paths]
inputs = processor(images=test_images, return_tensors="pt")
with torch.no_grad():
    for k in inputs:
        inputs[k] = inputs[k].cuda()
    output = model(**inputs)
print(output[0].shape)

torch.Size([1, 8, 4, 1601, 7680])


In [9]:
output[0].device

device(type='cuda', index=0)

In [20]:
import numpy as np
# 初始化存储所有特征的列表
all_image_features = []

# 设置批次大小
batch_size = 8

# 将模型转换为 bf16
model = model.bfloat16()

# 计算需要处理的批次数
num_batches = len(all_image_paths) // batch_size
if len(all_image_paths) % batch_size != 0:
    num_batches += 1

# 使用tqdm显示处理进度
for i in tqdm(range(num_batches)):
    # 获取当前批次的图像路径
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(all_image_paths))
    batch_image_paths = all_image_paths[start_idx:end_idx]
    
    # 读取并处理图像
    batch_images = [Image.open(x) for x in batch_image_paths]
    inputs = processor(images=batch_images, return_tensors="pt")
    
    # 将输入移到GPU
    with torch.no_grad():
        for k in inputs:
            inputs[k] = inputs[k].cuda()
        
        # 模型推理
        output = model(**inputs)
        
        # 将输出移到CPU并转换为numpy数组
        features = output[0].cpu().float().squeeze(0).numpy()
        all_image_features.append(features)

# 合并所有特征
all_image_features = np.concatenate(all_image_features, axis=1)

print("处理完成!")
print(f"特征形状: {all_image_features.shape}")

 12%|█▏        | 324/2632 [12:59<2:50:52,  4.44s/it]

: 

: 

In [6]:
import numpy as np
import torch
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from torch.utils.data import Dataset, DataLoader
import threading
from queue import Queue
import gc
import h5py

# 自定义数据集
class ImageDataset(Dataset):
    def __init__(self, image_paths, processor):
        self.image_paths = image_paths
        self.processor = processor
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # 读取图片
        image = Image.open(self.image_paths[idx])
        # 处理图片
        inputs = self.processor(images=[image], return_tensors="pt")
        # 释放图片内存
        image.close()
        return {k: v.squeeze(0) for k, v in inputs.items()}

# 数据预加载器
class DataPrefetcher:
    def __init__(self, loader):
        self.loader = iter(loader)
        self.stream = torch.cuda.Stream()
        self.preload()
        
    def preload(self):
        try:
            self.next_inputs = next(self.loader)
        except StopIteration:
            self.next_inputs = None
            return
        
        with torch.cuda.stream(self.stream):
            for k in self.next_inputs:
                # aspect_ratio_ids 保持为整数类型，其他转为 bfloat16
                if k == 'aspect_ratio_ids':
                    self.next_inputs[k] = self.next_inputs[k].cuda(non_blocking=True)
                else:
                    self.next_inputs[k] = self.next_inputs[k].cuda(non_blocking=True).bfloat16()
    
    def next(self):
        torch.cuda.current_stream().wait_stream(self.stream)
        inputs = self.next_inputs
        self.preload()
        return inputs
    
def process_images(model, projector, image_paths, save_dir, batch_size=8, num_workers=4, save_interval=1000):
    """
    处理图像并定期保存特征
    
    Args:
        model: 视觉模型
        projector: 降维投影器
        image_paths: 图像路径列表
        save_dir: 特征保存目录
        batch_size: 批次大小
        num_workers: 工作进程数
        save_interval: 多少张图像保存一次
    """
    # 确保模型和projector在GPU上且为bf16
    model = model.cuda().bfloat16().eval()
    projector = projector.cuda().bfloat16().eval()
    
    # 创建保存目录
    os.makedirs(save_dir, exist_ok=True)
    
    # 预热模型
    with torch.no_grad():
        dummy_inputs = processor(images=[Image.open(image_paths[0])], return_tensors="pt")
        dummy_input = {}
        for k, v in dummy_inputs.items():
            if k == 'aspect_ratio_ids':
                dummy_input[k] = v.cuda()
            else:
                dummy_input[k] = v.cuda().bfloat16()
        model(**dummy_input)
    
    # 创建数据集和数据加载器
    dataset = ImageDataset(image_paths, processor)
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        prefetch_factor=2
    )
    
    # 初始化特征列表和计数器
    all_image_features = []
    processed_count = 0
    current_batch = 0
    
    # 创建数据预加载器
    prefetcher = DataPrefetcher(loader)
    inputs = prefetcher.next()
    
    def save_features(features_tensor, image_paths_batch, batch_num):
        """保存特征到pt文件"""
        # 创建保存字典,包含特征和对应的图像路径
        save_dict = {
            'features': features_tensor,  # 保持bf16格式
            'image_paths': image_paths_batch
        }
        save_path = os.path.join(save_dir, f'features_batch_{batch_num}.pt')
        torch.save(save_dict, save_path)
        print(f"Saved features batch {batch_num} to {save_path}")

    # 主处理循环
    with torch.no_grad():
        with tqdm(total=len(image_paths)) as pbar:
            current_batch_paths = []
            
            while inputs is not None:
                # 记录当前批次的图像路径
                batch_size_actual = len(next(iter(inputs.values())))
                current_batch_paths.extend(image_paths[processed_count:processed_count + batch_size_actual])
                
                # 模型推理
                output = model(**inputs)
                
                # 使用projector降维并保持在GPU上的bf16格式
                features = output[0]  # 保持在GPU上
                features = projector(features).reshape(-1, features.shape[-3], features.shape[-2], 4096)
                
                # 直接添加tensor而不转换为numpy
                all_image_features.append(features.cpu())  # 只移到CPU,保持bf16格式
                processed_count += batch_size_actual

                # 获取下一批数据
                next_inputs = prefetcher.next()
                
                # 当需要保存时
                if processed_count >= (current_batch + 1) * save_interval or next_inputs is None:
                    # 合并当前批次的特征
                    current_features = torch.cat(all_image_features, dim=0)
                    
                    # 保存特征和对应的图像路径
                    save_features(current_features, current_batch_paths, current_batch)
                    
                    # 清空列表，准备下一批次
                    all_image_features = []
                    current_batch_paths = []
                    current_batch += 1
                    
                    # 清理内存
                    torch.cuda.empty_cache()
                    gc.collect()
                
                # 更新进度条
                pbar.update(batch_size_actual)
                
                # 更新inputs为下一批数据
                inputs = next_inputs
    
    print(f"处理完成! 总共处理了 {processed_count} 张图像，保存了 {current_batch} 个批次")
    return processed_count


# 使用示例
save_dir = "/mnt/data/CVPR2025/task1_data/images_features"  # 指定保存特征的目录
processed_count = process_images(
    model=model,
    projector=projector,
    image_paths=all_image_paths,
    save_dir=save_dir,
    batch_size=16,
    num_workers=16,
    save_interval=100
)

# 相应的加载函数也需要修改
def load_features(save_dir, batch_num):
    """加载指定批次的特征"""
    file_path = os.path.join(save_dir, f'features_batch_{batch_num}.pt')
    data_dict = torch.load(file_path)
    return data_dict['features'], data_dict['image_paths']

  0%|          | 96/21051 [00:22<1:20:00,  4.36it/s]

Saved features batch 0 to /mnt/data/CVPR2025/task1_data/images_features/features_batch_0.pt


  1%|          | 192/21051 [00:49<1:25:59,  4.04it/s]

Saved features batch 1 to /mnt/data/CVPR2025/task1_data/images_features/features_batch_1.pt


  1%|▏         | 288/21051 [01:16<1:26:01,  4.02it/s]

...

Saved features batch 209 to /mnt/data/CVPR2025/task1_data/images_features/features_batch_209.pt


100%|█████████▉| 21040/21051 [1:44:48<00:03,  3.48it/s]

Saved features batch 210 to /mnt/data/CVPR2025/task1_data/images_features/features_batch_210.pt


100%|██████████| 21051/21051 [1:44:53<00:00,  3.34it/s]

处理完成! 总共处理了 21051 张图像，保存了 211 个批次


In [14]:
def get_size_in_mb(array):
    """计算numpy数组的内存占用(MB)"""
    bytes_size = array.nbytes
    mb_size = bytes_size / (1024 * 1024)  # 转换为MB
    return mb_size

def get_size_info(array):
    """获取数组的详细信息"""
    shape = array.shape
    dtype = array.dtype
    size_mb = get_size_in_mb(array)
    size_gb = size_mb / 1024  # 转换为GB
    
    print(f"数组形状: {shape}")
    print(f"数据类型: {dtype}")
    print(f"内存占用: {size_mb:.2f} MB ({size_gb:.2f} GB)")
    print(f"元素个数: {array.size:,}")
    
# 计算all_image_features的大小
get_size_info(all_image_features)

数组形状: (100, 4, 1601, 7680)
数据类型: float32
内存占用: 18761.72 MB (18.32 GB)
元素个数: 4,918,272,000


In [8]:
for x in all_image_features:
    print(x.shape)

(16, 1, 4, 1601, 7680)
(16, 1, 4, 1601, 7680)
(16, 1, 4, 1601, 7680)
(16, 1, 4, 1601, 7680)
(16, 1, 4, 1601, 7680)
(16, 1, 4, 1601, 7680)
(4, 1, 4, 1601, 7680)
